In [1]:
from importlib import reload
import sys
stdout = sys.stdout
reload(sys)
# sys.setdefaultencoding('utf-8')
sys.stdout = stdout
# import sys
# stdout = sys.stdout
sys.path.append("..")
from lib import *
from sklearn.utils import shuffle

In [2]:
%mkdir -p data/research

In [3]:
import urllib.request as request
file_path = 'data/research/sonar.all-data.csv'
d_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data'
request.urlretrieve(d_url, file_path)

('data/research/sonar.all-data.csv',
 <http.client.HTTPMessage at 0x7f62eef45b38>)

In [3]:
sys.stdout = stdout
file_path = 'data/research/sonar.all-data.csv'

In [4]:
df = pd.read_csv(file_path, header=None)
df = shuffle(df)
df = df.reset_index()
depth = 3
benchmark = None
rows = list(range(1, 170))
features = list(range(3, 10))

In [5]:
random_seed = 678
cross_val_rnf_incremental(df, 5, 999, random_seed)

too many increments specified, running with the max possible...
<class 'numpy.int64'>
[81 98 24 62 35 21 90 23 60 92 19 22 66 50 63 42 78 73 28 61 75 36 71  9 91
 12 74 39 46  3 35 95 27 51 69 57 87 95 78 16 34 93 73 16 39 66 35 65 71 52
 47 35 14 85 73 88 89 47  8 23  0 69 77 19 23 89 80 70  8 95 93 28 94 16 99
 87 16 86 28 94 47 63 49 60 59 52 96  6 83 54 90 92 59 48 95 92 32 69 43 24]
score before incremental training: 0.7222222222222222
120
120
120
120
became right child
120
120
<class 'numpy.float64'>
error!:  117
[ 75  51  71  71  47  47  47  34  27  70  73  73  73  35  35  35  35  78
  78  52  52  12  22   9  16  16  16  16  88  32  63  63  65  77  57  98
  95  95  95  95  87  87  86  91  24  24  74  62  69  69  69  99  80  93
  93  21  81  85  94  94  28  28  28  92  92  92  66  66  54  49 117  62
   9  77  51  69  28  44  58  70 118  41  88  64  95  13  55 117  84  98
  57  11  45  37  87 113  12  84  66 106  95  63  21  49 117  31 106  10
  40  24   7  65  94 118   7  34  44 

UnboundLocalError: local variable 'members' referenced before assignment

In [ ]:
00